In [ ]:
import numpy as np
import sklearn
import pandas as pd

%matplotlib inline
from matplotlib import pyplot as plt

In [ ]:
df = pd.read_csv('datasets/FPU3_ML.csv')

In [ ]:
df.columns

In [ ]:
# Columns have extra spaces; let's correct that (and remove field, just to keep a fully numeric dataframe; why?)
df = pd.DataFrame(df.values[:, 1:], columns=df.columns.str.strip(' ')[1:], dtype='float64')

In [ ]:
df.info()

In [ ]:
df.describe().loc[: , ['FU1_X_calculated', 'FU2_X_calculated', 'FU3_X_calculated', 'FU4_X_calculated']]

In [ ]:
df.hist(['FU1_X_calculated', 'FU2_X_calculated', 'FU3_X_calculated', 'FU4_X_calculated', 'FU5_X_calculated'], bins=25)
df.hist(['FU1_Y_calculated', 'FU2_Y_calculated', 'FU3_Y_calculated', 'FU4_Y_calculated', 'FU5_Y_calculated'], bins=25)

In [ ]:
# Compute X and Y differences for each arm
for i in range(1, 6):
    for d in ['X', 'Y']:
        df['FU{}_d{}'.format(i, d)] = df['FU{}_{}_observed'.format(i, d)] - df['FU{}_{}_calculated'.format(i, d)]

In [ ]:
df.hist(['FU1_dX', 'FU2_dX', 'FU3_dX', 'FU4_dX', 'FU5_dX'], bins=25, figsize=(10, 10))

In [ ]:
# Drop some columns
cols_to_drop = []

for i in range(1, 6):
    for d in ['X', 'Y']:
        cols_to_drop.append('FU{}_{}_observed'.format(i, d))
        
df2 = df.drop(labels=cols_to_drop, axis=1, inplace=False)

In [ ]:
df2.corr(method='spearman').FU1_dX.abs().sort_values(ascending=False)

In [ ]:
plt.figure(figsize=(12, 6))
plt.scatter(df2.FU2_dX, df2.FU1_dX, s=df2.FU5_dX/1e2, c=df2.FU4_dX, alpha=0.3, label='FU5_dX')
plt.legend(loc=0)
plt.colorbar()

In [ ]:
plt.figure(figsize=(12, 6))
plt.scatter(df2.ha, df2.FU1_dX, c=df2.dec, alpha=1, label='FU5_dX')
plt.legend(loc=0)
plt.colorbar(label='Dec')

In [ ]:
cond = df2.FU1_dX > -30000

X = df2.loc[cond, ('ha', 'dec', 'airmass', 'alt', 'FU1_Delta_dec', 'FU1_Delta_ra', 'FU1_X_calculated', 'FU1_Y_calculated')]
t = df2.FU1_dX[cond]

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Split train test
X_train, X_test, t_train, t_test = train_test_split(X, t, test_size=0.2, random_state=1234)

# Scale
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

## Linear Regression

In [ ]:
from sklearn.linear_model import LinearRegression, HuberRegressor
from sklearn.metrics import mean_squared_error

lr = LinearRegression()
lr.fit(X_train, t_train)

# Metrics
y_train = lr.predict(X_train)

print('RMSE: {:.2f}'.format(np.sqrt(mean_squared_error(t_train, y_train))))

In [ ]:
res = t_train - y_train

plt.plot(X_train[:, 0], res, '.')

In [ ]:
from utils import hat_matrix
H = hat_matrix(X_train)

In [ ]:
plt.plot(np.diag(H), res, '.')
plt.axhline(0, color='r', ls=':')

**What out for those outliers!**


In [ ]:
# Remove high-leverage points
ind = np.diag(H) > 0.05

lr.fit(X_train[~ind], t_train[~ind])
y_train = lr.predict(X_train[~ind])

print('RMSE (train): {:.2f}'.format(np.sqrt(mean_squared_error(t_train[~ind], y_train))))



In [ ]:
plt.semilogx(np.diag(H)[~ind], y_train - t_train[~ind], '.')
plt.axhline(0, color='r', ls=':')

## Polynomial Regression

In [ ]:
from sklearn.preprocessing import PolynomialFeatures

poly = PolynomialFeatures(degree=2)

X_train_poly = poly.fit_transform(X_train)
X_test_poly = poly.transform(X_test)

pr = LinearRegression(fit_intercept=False)

pr.fit(X_train_poly, t_train)
y_train = pr.predict(X_train_poly)
y_test =  pr.predict(X_test_poly)
print('RMSE (train): {:.2f}'.format(np.sqrt(mean_squared_error(t_train, y_train))))
print('RMSE (test): {:.2f}'.format(np.sqrt(mean_squared_error(t_test, y_test))))

In [ ]:
X_train_poly.shape

## Decision Tree

In [ ]:
# Let's hit it harder
from sklearn.tree import DecisionTreeRegressor

dt = DecisionTreeRegressor(max_depth=7)
dt.fit(X_train, t_train)

# Metrics
y_train = dt.predict(X_train)
y_test = dt.predict(X_test)

print('RMSE (train): {:.2f}'.format(np.sqrt(mean_squared_error(t_train, y_train))))
print('RMSE (test): {:.2f}'.format(np.sqrt(mean_squared_error(t_test, y_test))))

In [ ]:
from sklearn.model_selection import GridSearchCV

gscv = GridSearchCV(dt, {'max_depth': range(2, 50)}, cv=5, scoring='neg_mean_squared_error')
gscv.fit(X_train, t_train)

print(gscv.best_params_)
print('Best RMSE (train): {:2f}'.format(np.sqrt(-gscv.best_score_)))
print('Best RMSE (test): {:2f}'.format(np.sqrt(mean_squared_error(gscv.best_estimator_.predict(X_test), t_test))))

## Random Forests

In [ ]:
from sklearn.ensemble import RandomForestRegressor

rfr = RandomForestRegressor(n_estimators=500)
rfr.fit(X_train, t_train)

y_train = rfr.predict(X_train)
y_test =  rfr.predict(X_test)
print('RMSE (train): {:.2f}'.format(np.sqrt(mean_squared_error(t_train, y_train))))
print('RMSE (test): {:.2f}'.format(np.sqrt(mean_squared_error(t_test, y_test))))

In [ ]:
RandomForestRegressor?

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint

rscv = RandomizedSearchCV(rfr, {'max_depth': randint(2, 20), 'n_estimators': randint(100, 500)}, 
                          n_iter=20, cv=5, scoring='neg_mean_squared_error')
rscv.fit(X_train, t_train)

print(rscv.best_params_)
print('Best RMSE (train): {:2f}'.format(np.sqrt(-rscv.best_score_)))
print('Best RMSE (test): {:2f}'.format(np.sqrt(mean_squared_error(rscv.best_estimator_.predict(X_test), t_test))))

In [ ]:
y_train = rscv.best_estimator_.predict(X_train)
y_test = rscv.best_estimator_.predict(X_test)

In [ ]:
plt.scatter(t_train, y_train, label='train data')
plt.scatter(t_test, y_test, label='test data')
plt.legend(loc=0)
plt.xlabel('FU1_dX [target]', fontsize=16)
plt.ylabel('FU1_dX [predicted]', fontsize=16)
plt.savefig('/Users/rodrigo/EXOML/extra/plots/RF_run1.pdf')

I am tempted to repeat everything for points >-30000